# 1. Imports

In [ ]:
import numpy as np
import pandas as pd 
import os
import seaborn as sns
import shap

from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)

seed = 42


# 2. Functions

In [ ]:
def WMAE(y_true, y_pred, df):
    """calculates the WMAE metric as specified by the competition"""
    weights = df.IsHoliday.apply(lambda x: 5 if x != 0 else 1)
    return np.round( np.sum( weights*abs( y_true - y_pred ))/( np.sum( weights )), 2)

# 3. Loading Data

Gathering all the data available 

In [ ]:
# train data
path = '../input/walmart-recruiting-store-sales-forecasting/'
train = pd.read_csv(path+'train.csv.zip')
stores = pd.read_csv(path+'stores.csv')
features = pd.read_csv(path+'features.csv.zip')

# test data
test = pd.read_csv(path+'test.csv.zip')

# 4. Joining Tables

## 4.1. Filling missing dates


Some Store Departments have no data for some weeks so we are going to input these instances with zero sales. More details about this is available in the EDA Section.

In [ ]:
# Creating new index
dates = train.Date.sort_values().unique()
depts = train.Dept.sort_values().unique()
strs = train.Store.sort_values().unique()

fill_index = pd.MultiIndex.from_product([dates, strs, depts],
                           names=['Date','Store','Dept'])

# Creating holidays dataframe
holidays = train.query('Store==1 & Dept==1')[['Date','IsHoliday']]


# Filling with reindex
train_new = train.set_index(['Date','Store','Dept']).Weekly_Sales.reindex(fill_index, fill_value = 0)
train_new = train_new.to_frame()
train_new.reset_index(inplace=True)

# merging back with holidays

train_new = train_new.merge(holidays, on='Date', how = 'left')


## 4.2. Merge

Merging the new train dataset with Stores and Features 

In [ ]:
data = train_new.merge(stores, on = 'Store', how = 'left').merge(features.drop(columns=['IsHoliday']), on =['Store','Date'], how = 'left')

#data.IsHoliday_x.equals(data.IsHoliday_y)  # columns are equal so one can be dropped

# Saving merged data
data.to_csv("data.csv")

# 5. EDA

The Exploratory Data analysis is done so we can familiarize ourselves with the data and gain insight and intuition about the problem. This comes in hand to understand future model behavior or eventual unexpected results. In this phase, is essential to follow up with business experts and other stakeholders.

## Noteworthy points 

- There are departments with missing dates in between entries, it may be the case that zero sales are not being accounted for. In a real situation, it is important to understand if this is the case or not. In this notebook, we will consider that missing values are indicatives of zero sales; therefore, this information might be useful for the model. We will input zero sales where there are missing department data.
- We found some negative Weekly Sales in the database. Again, it is important to understand, with a business expert or someone with experience with these data, which situations may lead to this. For this notebook, we will consider negative sales as refunds, and since the challenge asks for Sales prediction, we will replace negative values with 0. This might not be a big problem since negative values represent less then 0.4% of instance entries, but still, it is important to understand if the model is expected to predict negative values.

In [ ]:
# Describe
data.describe().T

# negative values for Sales

In [ ]:
# Types
print(data.head(),'\n\n',data.dtypes) 

# datatypes are all good but cam be optimized to reduce memory usage

In [ ]:
# Check for Nulls
print(data.count(),'\n\n',data.isna().sum()) 

# null values only on markdown columns

In [ ]:
# Stores entries
# data['Store'].value_counts()  

# some stores have slightly more entries than others...
# this was fixed using imputation 

In [ ]:
data.groupby(['Store','Dept']).agg(dates = ('Date', 'count'))

# all departments now have the same number of entries

In [ ]:
# Analisis Ideas

# Total Sales by department
# Total Sales by store
# Pairplot

# 6. Feature Engineering

- OK - Parse Dates to label encoding (months and weeks of the year)
- OK - Encode variable "Type" (one hot)
- Create variables #weeks before holiday
- OK - Input missing Dept instances as zero sales (treating missing data as zero sales)
- OK - What are negative sales? (1285 entries in the test database  = 0.3% max of -4988.94 and total of -88161.56 adding up to -0,000013086% of the total sales) replace with zero?
- OK - Fill Nulls in markdown variables

In [ ]:
# Replacing Negative Sales with 0
data[data.Weekly_Sales < 0 ].Weekly_Sales = 0

In [ ]:
# FillNA markdown variables with -9999
data.fillna(-9999,inplace = True)

In [ ]:
# OneHot encoding variable store type
data = pd.get_dummies(data,columns=['Type'])

In [ ]:
# parse data into month and weekofyear columns
data['Month'] = data.Date.apply(lambda x : datetime.strptime(str(x),'%Y-%m-%d').month)
data['WeekofYear'] = data.Date.apply(lambda x : datetime.strptime(str(x),'%Y-%m-%d').isocalendar()[1])


In [ ]:
# holidays weeks
data.query('IsHoliday == True').WeekofYear.unique()

In [ ]:
data['IsHoliday'] = data.query('IsHoliday == True').WeekofYear
data['IsHoliday_1'] = data.query('WeekofYear in (5, 35, 46, 51)').WeekofYear

In [ ]:
data = pd.concat([pd.get_dummies(data,columns=['IsHoliday','IsHoliday_1'],prefix=['Holiday','Week_Before_Holiday']),data['IsHoliday']], axis = 1)

In [ ]:
data.head()

In [ ]:
# reducing memory usage

print(f'data memory usage : {data.memory_usage(deep = True).sum() / 1024**2} Mb')

In [ ]:
ints = data.select_dtypes(include='int').columns.values.tolist()
floats = data.select_dtypes(include='float').columns.values.tolist()


In [ ]:
for i in ints:
    data[i] = pd.to_numeric(data[i],downcast = 'integer')
    
for i in floats:
    data[i] = pd.to_numeric(data[i],downcast = 'float')

In [ ]:
print(f'data memory usage after optimizing dtypes : {data.memory_usage(deep = True).sum() / 1024**2} Mb')

# 7. Modeling

It is good practice to start with simpler models and gradually build up complexity. Simpler models are easier to interpret and can give good insight about the importance and influence of variables in the final result. Also, it is good to have an initial model ready and running as soon as possible so we can get a baseline of performance for the next iterations. In this Notebook, however, I will skip this phase since, as shown in previous sections, the dependent variable is highly non-linear. This means that simpler models such as linear regression will perform poorly and probably won't be of much use for interpretability.

I choose to jump right into a random forest model, mainly because this algorithm can deal with non-linear data with ease. Random Forests also can handle data without standardizing or normalizing, and we don't have to worry about the imputation of cyclical time variables such as month and week of the year. This simplifies the preprocessing and saves us some time, also the model will give us a pretty good approximation of an "optimum" model right away.

## Noteworthy points
- Out of time validation set
- Started with a sample in the randon search so we can make some initial parameters tunning
- Make sure of not going too deep with the trees to avoid overfiting, oot set evaluation also helps
- Variable importances



In [ ]:
data.fillna(0,inplace = True)

# out of time validation set
oot_data = data.query("Date >= '2012-07-01'")

oot_X = oot_data.drop(columns = ['Date','Weekly_Sales'])
oot_y = oot_data.Weekly_Sales

# in time validation set
it_data = data.query("Date < '2012-07-01'")

data_sample = it_data.sample(frac=.5, random_state = seed)

X = data_sample.drop(columns = ['Date','Weekly_Sales'])
y = data_sample.Weekly_Sales

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

In [ ]:
X_train.shape,y_train.shape,X_test.shape,oot_X.shape

In [ ]:
rf = RandomForestRegressor(random_state=seed, n_jobs=-1)
random_grid = {'bootstrap': [True, False],
               'max_depth': [5, 10, 20, 30, 40, 50, 60],
               'max_features': ['auto','sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10],
               'n_estimators': [200, 400, 600]}


random_grid = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3,verbose=2, random_state=seed)

# Fit the random search model
random_grid.fit(X_train, y_train)

print(random_grid.best_params_)

rand_reg = random_grid.best_estimator_.fit(X_train, y_train)

In [ ]:
imp_rf = pd.DataFrame(rand_reg.feature_importances_).transpose()
imp_rf.columns = X_train.columns
imp_rf = imp_rf.transpose().sort_values(0)
imp_rf.columns = ['rf_importance']

imp_rf.plot.barh()

In [ ]:
# fine tunning

# {'n_estimators': 600, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 40, 'bootstrap': True}

rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=40, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rand_reg = rf.fit(X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0']),y_train)

In [ ]:
print("train MAE: {:.2f}".format(mean_absolute_error(y_train,rand_reg.predict(X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])))))
print("test MAE: {:.2f}".format(mean_absolute_error(y_test,rand_reg.predict(X_test.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])))))
print("OOT_test MAE: {:.2f}".format(mean_absolute_error(oot_y,rand_reg.predict(oot_X.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])))))
print("train WMAE: {:.2f}".format(WMAE(y_train,rand_reg.predict(X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])), X_train)))
print("test WMAE: {:.2f}".format(WMAE(y_test,rand_reg.predict(X_test.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])), X_test)))
print("OOT_test WMAE: {:.2f}".format(WMAE(oot_y,rand_reg.predict(oot_X.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])), oot_X)))

In [ ]:
print("train MAE: {:.2f}".format(mean_absolute_error(y_train,rand_reg.predict(X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0'])))))
print("test MAE: {:.2f}".format(mean_absolute_error(y_test,rand_reg.predict(X_test.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0'])))))
print("OOT_test MAE: {:.2f}".format(mean_absolute_error(oot_y,rand_reg.predict(oot_X.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0'])))))
print("train WMAE: {:.2f}".format(WMAE(y_train,rand_reg.predict(X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0'])), X_train)))
print("test WMAE: {:.2f}".format(WMAE(y_test,rand_reg.predict(X_test.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0'])), X_test)))
print("OOT_test WMAE: {:.2f}".format(WMAE(oot_y,rand_reg.predict(oot_X.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0'])), oot_X)))

In [ ]:
X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0']).head()

In [ ]:
explainer = shap.TreeExplainer(rand_reg, approximate = True)
rf_shap_values = explainer.shap_values(X_test.iloc[0:1000,:])
shap.summary_plot(rf_shap_values, X_test.iloc[0:1000,:])

# 8. Validation

# 9. Conclusion

In [ ]:
data = test.merge(stores, on = 'Store', how = 'left').merge(features.drop(columns=['IsHoliday']), on =['Store','Date'], how = 'left')

# FillNA markdown variables with -9999
data.fillna(-9999,inplace = True)

# OneHot encoding variable store type
data = pd.get_dummies(data,columns=['Type'])

# parse data into month and weekofyear columns
data['Month'] = data.Date.apply(lambda x : datetime.strptime(str(x),'%Y-%m-%d').month)
data['WeekofYear'] = data.Date.apply(lambda x : datetime.strptime(str(x),'%Y-%m-%d').isocalendar()[1])

# holidays weeks
data.query('IsHoliday == True').WeekofYear.unique()

data['IsHoliday'] = data.query('IsHoliday == True').WeekofYear
data['IsHoliday_1'] = data.query('WeekofYear in (5, 35, 46, 51)').WeekofYear

data = pd.concat([pd.get_dummies(data,columns=['IsHoliday','IsHoliday_1'],prefix=['Holiday','Week_Before_Holiday']),data['IsHoliday']], axis = 1)

ints = data.select_dtypes(include='int').columns.values.tolist()
floats = data.select_dtypes(include='float').columns.values.tolist()


In [ ]:
data.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']).head()

In [ ]:
X_train.drop(columns=['Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Holiday_36.0','Week_Before_Holiday_35.0']).head()

In [ ]:
data.fillna(0,inplace = True)
data['Weekly_Sales'] = rand_reg.predict(data.drop(columns = ['Date','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']))

In [ ]:
test.head()

In [ ]:
final = data
final['id'] = data['Store'].map(str) + '_' + data['Dept'].map(str) + '_' + data['Date'].map(str)
final['Weekly_Sales'] = data['Weekly_Sales']

final = final[['id','Weekly_Sales']]

In [ ]:
final.to_csv('submission.csv',index=False)